In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

from pose_estimation import PoseData, PoseDataNPZ, PoseDataNPZTorch, COLOR_PALETTE

WORKDIR = f"{os.getcwd()}/.."
DATA_FOLDER = os.path.join(WORKDIR, "data_folder")
DATASET_NPZ_PATH = os.path.join(DATA_FOLDER, "dataset_npz")


In [2]:
dataset = None
dataset = PoseDataNPZ(DATASET_NPZ_PATH)


Presumed Preloaded NPZ Dataset: /Users/armanommid/Code/CSE/CSE275/HW2/XSwinDiffusion/../data_folder/dataset_npz


In [ ]:
PoseData(os.path.join(DATA_FOLDER, "training_data"), os.path.join(DATA_FOLDER, "models")).npz(
    os.path.join(DATA_FOLDER, "dataset_npz")
)
PoseData(os.path.join(DATA_FOLDER, "testing_data"), os.path.join(DATA_FOLDER, "models")).npz(
    os.path.join(DATA_FOLDER, "dataset_npz_test")
)


In [4]:
for i, key in enumerate(dataset.keylist):
    label = dataset[key]["label"]
    # object_ids = [object_id for object_id in np.unique(label) if object_id < 79]
    print(key)


(1, 7, 13)
(2, 15, 1)
(2, 20, 8)
(2, 89, 19)
(1, 90, 18)
(1, 97, 11)
(2, 50, 1)
(2, 65, 8)
(1, 59, 7)
(2, 57, 17)
(2, 76, 2)
(2, 87, 20)
(1, 143, 12)
(2, 33, 2)
(1, 15, 12)
(2, 127, 15)
(2, 96, 7)
(2, 59, 12)
(1, 40, 13)
(1, 224, 19)
(2, 80, 15)
(1, 99, 14)
(1, 9, 16)
(3, 22, 10)
(1, 44, 2)
(3, 25, 19)
(1, 33, 16)
(1, 62, 1)
(2, 153, 19)
(1, 57, 8)
(1, 61, 36)
(1, 68, 12)
(2, 71, 13)
(1, 27, 1)
(1, 12, 8)
(1, 66, 17)
(1, 82, 4)
(2, 24, 12)
(3, 155, 28)
(1, 139, 27)
(2, 98, 8)
(3, 77, 11)
(2, 47, 12)
(2, 52, 3)
(2, 17, 3)
(2, 211, 47)
(1, 57, 37)
(1, 38, 6)
(2, 31, 0)
(2, 205, 26)
(2, 41, 9)
(1, 87, 14)
(2, 74, 0)
(1, 89, 11)
(2, 90, 10)
(3, 136, 14)
(2, 94, 5)
(1, 184, 15)
(2, 97, 19)
(1, 80, 21)
(2, 99, 20)
(2, 12, 13)
(1, 50, 16)
(2, 49, 17)
(1, 23, 13)
(1, 73, 9)
(1, 46, 0)
(2, 61, 16)
(1, 78, 17)
(1, 36, 9)
(2, 33, 36)
(1, 25, 3)
(2, 4, 11)
(2, 3, 18)
(2, 69, 5)
(1, 60, 3)
(2, 89, 0)
(1, 80, 6)
(2, 118, 24)
(1, 76, 12)
(2, 34, 17)
(1, 151, 12)
(2, 95, 20)
(2, 37, 6)
(2, 9, 5)
(1, 5